<a href="https://colab.research.google.com/github/PedrolyraC/electromechanical-conversion/blob/main/electromechanical_conversion_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [43]:
def calculate_a_and_b_geometric_section(geometric_section: float):
    if geometric_section > 25:
        return geometric_section/5
    return geometric_section/4

In [3]:
def core_geometric_section_1(magnetic_section: float):
    return magnetic_section*1.1

In [4]:
def core_magnetic_section(length: float, width: float)->float:
    return core_geometric_section(length, width)/1.1

In [5]:
def core_geometric_section(length: float, width: float)->float:
    return length*width

In [6]:
def core_magnetic_section1(length: float, width: float)->float:
    return 1.1*core_geometric_section(length, width)

In [7]:
def cooper_section(n1:float,s1:float,n2:float,s2:float):
    return n1*s1 + n2*s2

In [8]:
def conductor_section(current, potency)->float:
    if potency == 500:
        return current/3
    if potency >= 500 and potency <= 1000:
        return current/2.5
    if potency >= 1000 and potency <= 3000:
        return current/2

    return 'Invalid value'

In [46]:
def iron_core_weight(geometric_section:float):
    b = calculate_a_and_b_geometric_section(geometric_section)

    if(geometric_section>25):
        return 1.580*b
    return 1.000*b

In [47]:
def calculate_lm(core_gs):
    b = calculate_a_and_b_geometric_section

    if(core_gs>25):
        return (2*5 + 2*b + 0.5*3.14*5)
    return (2*4 + 2*b + 0.5*3.14*4)

In [11]:
def cooper_weight(geometric_section:float,n1:float,s1:float,n2:float,s2:float):
    lm=calculate_lm(geometric_section)
    cooper_sec=cooper_section(n1,s1,n2,s2)

    return (cooper_sec)/100*lm*9

In [12]:
def copper_core_weight(
    first_tension: float,
    second_tension: float,
    first_potency: float,
    second_potency: float,
    frequency: float,
    is_long_cable,
    is_two_primary_circuits: bool=False,
    is_two_secondary_circuits: bool=False
    )->float:
    geometric_section = calculate_magnectic_section_core(second_potency, frequency, is_long_cable, is_two_primary_circuits, is_two_secondary_circuits)
    aux = geometric_section
    a = round(math.sqrt(aux*1.1))
    b = geometric_section/a
    average_length = 2*a + 2*b + 0.5*a*math.pi
    s1 = conductor_section(first_tension, first_potency)
    s2 = conductor_section(second_tension, second_potency)
    n1 = first_tension*1.1
    n2 = second_tension*1.1
    section_copper = n1*s1 + n2*s2

    return (section_copper/100)*average_length*9

In [22]:
def calculate_turns_number_1(frequency: float, tension: float, core_ms: float):
    if(frequency == 50):
        return tension*(40/core_ms)
    return tension*(33.5/core_ms)

In [30]:
def exec_possibility_1(n1:float, n2:float, s1:float, s2: float,geometric_section:float):
    possibility=2400/cooper_section(n1,s1,n2,s2)

    if(geometric_section >25):
        possibility= 3750/cooper_section(n1,s1,n2,s2)
        return (f'Possible to project the transformer. Coeficient: {possibility}' if(possibility>=3)
                else f'Impossible to project this transformer. Please, reproject the transformer with a bigger core. Coeficient:{possibility}')
    return (f'Possible to project the transformer. Coeficient: {possibility}' if(possibility>=3)
            else f'Impossible to project this transformer. Please, reproject the transformer with a bigger core. Coeficient:{possibility}')

In [15]:
def calculate_turns_number(
    tension:float,
    sm:float,
    frequency:float,
    bm:float = 11300
    )->float:
    return (tension/sm)*(1e8/(4.44*bm*frequency))

In [31]:
def calculate_magnectic_section_core(
    potency: float,
    frequency: float,
    is_long_cable: bool,
    is_two_primary_circuits: bool=False,
    is_two_secondary_circuits: bool=False
    )->float:
    standard_cables = 7.5*(math.sqrt(potency/frequency))
    long_cables = 6*(math.sqrt(potency/frequency))

    if is_two_primary_circuits is False and is_two_secondary_circuits is False:
        if is_long_cable:
            return long_cables
        return standard_cables

    if is_two_primary_circuits is True and is_two_secondary_circuits is False:
        standard_cables = 7.5*(math.sqrt(1.25*potency/frequency))
        long_cables = 6*(math.sqrt(1.25*potency/frequency))

        if is_long_cable:
            return long_cables
        return standard_cables

    if is_two_primary_circuits is True and is_two_secondary_circuits is True:
        standard_cables = 7.5*(math.sqrt(1.5*potency/frequency))
        long_cables = 6*(math.sqrt(1.5*potency/frequency))

        if is_long_cable:
            return long_cables
        return standard_cables

    print('Invalid option')
    return [0, 0]

In [44]:
def meroshow (frequency: float, second_potency:float, first_tension: float, second_tension:float):
    first_potency = second_potency*1.1
    first_current = first_potency/first_tension
    second_current = second_potency/second_tension
    bm = 11300

    s1 = conductor_section(first_current, second_potency)
    s2 = conductor_section(second_current, second_potency)

    magnetic_section = calculate_magnectic_section_core(first_potency, frequency, True)
    geometric_section = core_geometric_section_1(magnetic_section)

    if geometric_section > 25:
        a = 5
    else:
        a = 4
    b = calculate_a_and_b_geometric_section(geometric_section)

    core_gs = core_geometric_section(a, b)
    core_ms = core_magnetic_section(a, b)

    n1 = calculate_turns_number_1(frequency, first_tension, core_ms)
    n2 = calculate_turns_number_1(frequency,  second_tension, core_ms)*1.1
    possibility = exec_possibility_1(n1,n2,s1,s2,core_gs)
    iron_weight = iron_core_weight(core_gs)
    section_cooper= cooper_section(n1,s1,n2,s2)

    print("first_potency", first_potency)
    print("first_current", first_current)
    print("second_current", second_current)
    print("bm", bm)
    print("s1: ", s1)
    print("s2: ", s2)
    print("magnetic_section: ", magnetic_section)
    print("geometric_section: ", geometric_section)
    print("core_ms: ", core_ms)
    print("core_gs: ", core_gs)
    print("n1: ", n1)
    print("n2: ", n2)
    print(possibility)
    print("iron_core_weight: ",iron_weight)
    print("cooper Section:", section_cooper, "mm²")


In [45]:
def analyze_table(bitola_wire: float):
    database = pd.read_excel('awg.xlsx')
    database = database.sort_values('section').reset_index(drop=True)
    sections = database['section'].values
    idx = np.searchsorted(sections, bitola_wire, side='right')

    if idx < len(sections):
        nearest_awg = database.iloc[idx]['awg']
        return nearest_awg
    else:
        return database.iloc[-1]['awg']

In [42]:
meroshow(50,1000,220,24)

first_potency 1100.0
first_current 5.0
second_current 41.666666666666664
bm 11300
s1:  2.0
s2:  16.666666666666664
magnetic_section:  28.142494558940577
geometric_section:  30.956744014834637
core_ms:  28.142494558940577
core_gs:  30.956744014834637
n1:  312.69438398822865
n2:  37.52332607858744
Impossible to project this transformer. Please, reproject the transformer with a bigger core. Coeficient:2.9981350737507717
iron_core_weight:  9.782331108687746
cooper Section: 1250.7775359529146 mm²
